In [1]:
import csv
import polars as pl
import yfinance as yf

class Data_Engineering():
    def __init__(self):
        self.value_asset_data = dict()
        self.price_data = dict()
        self.amount_asset_data = dict()

    def value_asset_data_retriever(self, path_file, asset_line_end, asset_line_start = 3):
        
        with open(path_file,   mode  = 'r') as f: # Apri il file; utilizza csv.reader perché gestisce i decimali con la virgola
            reader = csv.reader(f)
            
            for _ in range(asset_line_start): # Skippa le righe fino ad "asset_line_start" nel file excel
                next(reader)

            for i, values in enumerate(reader): # Assegna al dizionario asset_data il primo valore di ogni riga e quelli relativi ad ogni mese
                if i == asset_line_end - asset_line_start: # Considera tutte le righe entro "asset line end"
                    break
                self.value_asset_data[values[0]] = list(map(lambda x : int(x.replace(',', '')), values[3:15])) # Assegna a ogni ticker una lista che contiene il valore degli asset. Ad ogni valore è rimossa la virgola e "int"-erizzata

    def price_data_retriever(self, year):
        tickers = list(self.value_asset_data.keys()) # Ottieni la lista di tickers dal file degli asset
        for ticker in tickers: # Per ogni ticker, ottieni i dati giornalieri di chiusura del prezzo, quindi campiona i dati per ottenere i prezzi di chiusura degli ultimi giorni
            self.price_data[ticker] = yf.Ticker('VT').history(start = str(year)+'-01-01', end = str(year+1)+'-01-01')['Close'].resample('ME').last().tolist()

    def amount_data_calculator(self):
        for key in self.value_asset_data:
            self.amount_asset_data[key] = [value / price for value, price in zip(self.value_asset_data[key], self.price_data[key])]
        
data_2023 = Data_Engineering()
data_2023.value_asset_data_retriever('source_files/RIP Net Worth (mr.rip_nw) - 2023.csv', 13)
data_2023.price_data_retriever(2023)
data_2023.amount_data_calculator()

In [2]:
value_2023 = pl.from_dict(data_2023.value_asset_data)
price_2023 = pl.from_dict(data_2023.price_data)
amount_2023 = pl.from_dict(data_2023.amount_asset_data)

In [3]:
print(value_2023, price_2023, amount_2023)

shape: (12, 10)
┌────────┬────────┬────────┬──────┬───┬───────┬──────┬───────┬──────┐
│ VT     ┆ VYM    ┆ VYMI   ┆ ERUS ┆ … ┆ VBR   ┆ SONY ┆ INTEL ┆ CHEK │
│ ---    ┆ ---    ┆ ---    ┆ ---  ┆   ┆ ---   ┆ ---  ┆ ---   ┆ ---  │
│ i64    ┆ i64    ┆ i64    ┆ i64  ┆   ┆ i64   ┆ i64  ┆ i64   ┆ i64  │
╞════════╪════════╪════════╪══════╪═══╪═══════╪══════╪═══════╪══════╡
│ 227311 ┆ 72001  ┆ 250770 ┆ 40   ┆ … ┆ 69284 ┆ 8946 ┆ 14130 ┆ 3050 │
│ 220084 ┆ 69472  ┆ 254856 ┆ 40   ┆ … ┆ 67720 ┆ 8358 ┆ 12465 ┆ 3300 │
│ 234830 ┆ 68575  ┆ 253995 ┆ 40   ┆ … ┆ 63504 ┆ 9065 ┆ 16335 ┆ 1430 │
│ 247510 ┆ 69446  ┆ 262933 ┆ 40   ┆ … ┆ 62896 ┆ 8967 ┆ 15530 ┆ 1360 │
│ 244516 ┆ 65884  ┆ 253441 ┆ 40   ┆ … ┆ 60744 ┆ 9370 ┆ 15720 ┆ 1500 │
│ …      ┆ …      ┆ …      ┆ …    ┆ … ┆ …     ┆ …    ┆ …     ┆ …    │
│ 263871 ┆ 86152  ┆ 271975 ┆ 40   ┆ … ┆ 67380 ┆ 8319 ┆ 17570 ┆ 2910 │
│ 186360 ┆ 98154  ┆ 266127 ┆ 40   ┆ … ┆ 63796 ┆ 8241 ┆ 17775 ┆ 4330 │
│ 180920 ┆ 95428  ┆ 258129 ┆ 40   ┆ … ┆ 60708 ┆ 8305 ┆ 18250 ┆ 3480 │
│ 19